In [1]:
!pip install -q pyomo
from pyomo.environ import *
import numpy as np

In [2]:
cost = [[627 ,617 ,1270, 2072, 1415 ,2943 ,1064, 4265],
[159 ,1432, 351, 3372, 237, 3373, 1456, 5059],
[609 ,1943, 77 ,3365, 298, 3021 ,1484, 4803 ],
[2143 ,671 ,3876 ,537 ,1902 ,1941 ,1682 ,3348],
[2742 ,2089, 1931, 2199, 1226, 739 ,691 ,1128 ],
[2053 ,860 ,2995, 68 ,2069 ,2065 ,993 ,3342],
[656 ,817, 1245, 2141, 1534, 3351, 1137, 3887],
[195 ,1862 ,1331, 2921 ,402 ,2825 ,563 ,4441]]

In [3]:
cost = np.array(cost)
def bmatrix(a):
    """Returns a LaTeX bmatrix

    :a: numpy array
    :returns: LaTeX bmatrix as a string
    """
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    return '\n'.join(rv)

A = np.array(cost)
print(bmatrix(A) + '\n')

# B = np.array([[1.2], [3.7], [0.2]])
# print bmatrix(B) + '\n'

# C = np.array([1.2, 9.3, 0.6, -2.1])
# print bmatrix(C) + '\n'

\begin{bmatrix}
  627 & 617 & 1270 & 2072 & 1415 & 2943 & 1064 & 4265\\
  159 & 1432 & 351 & 3372 & 237 & 3373 & 1456 & 5059\\
  609 & 1943 & 77 & 3365 & 298 & 3021 & 1484 & 4803\\
  2143 & 671 & 3876 & 537 & 1902 & 1941 & 1682 & 3348\\
  2742 & 2089 & 1931 & 2199 & 1226 & 739 & 691 & 1128\\
  2053 & 860 & 2995 & 68 & 2069 & 2065 & 993 & 3342\\
  656 & 817 & 1245 & 2141 & 1534 & 3351 & 1137 & 3887\\
  195 & 1862 & 1331 & 2921 & 402 & 2825 & 563 & 4441\\
\end{bmatrix}



In [4]:
model = ConcreteModel()
row = np.arange(8)
col = np.arange(8)
m = [175,400, 550, 70, 650, 100, 65, 85]
w = [185,350,405,280,105,75,425,270]
model.x = Var(row,col, domain = NonNegativeIntegers)
model.obj = Objective(expr = sum(model.x[i,j]*cost[i][j] for i in row  for j in col), sense = minimize)
model.constraint = ConstraintList()
[model.constraint.add(expr = sum(model.x[i,j] for i in row) >= m[j]) for j in col ]
[model.constraint.add(expr = sum(model.x[i,j] for j in col) <= w[i]) for i in row ]
model.pprint()

4 Set Declarations
    constraint_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   16 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   64 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)}
    x_index_0 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        

In [5]:
# install cbc solver
!apt-get install -y -qq coinor-cbc
# using solverfactory to solve above constructed pyomo model
optCBC = SolverFactory('cbc')

# solving your pipeModel using cbc solver
results = optCBC.solve(model)

In [6]:
print(results)


Problem: 
- Name: unknown
  Lower bound: 1166465.0
  Upper bound: 1166465.0
  Number of objectives: 1
  Number of constraints: 16
  Number of variables: 64
  Number of binary variables: 0
  Number of integer variables: 64
  Number of nonzeros: 64
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.0222933292388916
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [7]:
model.obj()

1166465.0

In [8]:
for i in row:
  for j in col:
    if model.x[i,j].value >0:
      print("W{i}->M{j}:".format(i=i+1,j=j+1),model.x[i,j].value)

W1->M2: 185.0
W2->M5: 350.0
W3->M3: 375.0
W3->M5: 30.0
W4->M2: 200.0
W4->M6: 80.0
W5->M6: 20.0
W5->M8: 85.0
W6->M4: 70.0
W6->M7: 5.0
W7->M1: 175.0
W7->M2: 15.0
W7->M3: 175.0
W7->M7: 60.0
W8->M5: 270.0


In [9]:
import pandas as pd
dflist = np.zeros((len(row), len(col)))


In [10]:
dflist

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [11]:
for i in row:
  for j in col:
    dflist[i][j] = model.x[i,j].value


In [12]:
dflist

array([[  0., 185.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0., 350.,   0.,   0.,   0.],
       [  0.,   0., 375.,   0.,  30.,   0.,   0.,   0.],
       [  0., 200.,   0.,   0.,   0.,  80.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,  20.,   0.,  85.],
       [  0.,   0.,   0.,  70.,   0.,   0.,   5.,   0.],
       [175.,  15., 175.,   0.,   0.,   0.,  60.,   0.],
       [  0.,   0.,   0.,   0., 270.,   0.,   0.,   0.]])

In [13]:
df = pd.DataFrame(dflist,columns = ['M1','M2','M3','M4','M5','M6','M7','M8'] , index = ['W1','W2','W3','W4','W5','W6','W7','W8'])

In [14]:
df

,M1,M2,M3,M4,M5,M6,M7,M8
W1,0.0,185.0,0.0,0.0,0.0,0.0,0.0,0.0
W2,0.0,0.0,0.0,0.0,350.0,0.0,0.0,0.0
W3,0.0,0.0,375.0,0.0,30.0,0.0,0.0,0.0
W4,0.0,200.0,0.0,0.0,0.0,80.0,0.0,0.0
W5,0.0,0.0,0.0,0.0,0.0,20.0,0.0,85.0
W6,0.0,0.0,0.0,70.0,0.0,0.0,5.0,0.0
W7,175.0,15.0,175.0,0.0,0.0,0.0,60.0,0.0
W8,0.0,0.0,0.0,0.0,270.0,0.0,0.0,0.0


In [15]:
model1 = ConcreteModel()
row = np.arange(8)
col = np.arange(8)
m = [175,400, 550, 70, 650, 100, 65, 85]
w = [185,350,405,280,105,75,425,270]
model1.x = Var(row,col, domain = NonNegativeIntegers)
model1.obj = Objective(expr = sum(model1.x[i,j]*cost[i][j] for i in row  for j in col), sense = minimize)
model1.constraint = ConstraintList()
[model1.constraint.add(expr = sum(model1.x[i,j] for i in row) >= m[j]) for j in col ]
[model1.constraint.add(expr = sum(model1.x[i,j] for j in col) <= w[i]) for i in row ]
model1.constraint.add(expr = model1.x[5,3] == 0)
model1.constraint.add(expr = model1.x[4,7] == 0)
model1.pprint()

4 Set Declarations
    constraint_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   18 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   64 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)}
    x_index_0 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members


In [16]:
# using solverfactory to solve above constructed pyomo model
optCBC = SolverFactory('cbc')

# solving your pipeModel using cbc solver
results = optCBC.solve(model1)

In [17]:
print(model1.obj())

1289875.0


In [20]:
for i in row:
  for j in col:
    if model1.x[i,j].value >0:
      print("W{i}->M{j}:".format(i=i+1,j=j+1),model1.x[i,j].value)

dflist = np.zeros((len(row), len(col)))
for i in row:
  for j in col:
    dflist[i][j] = model1.x[i,j].value
df = pd.DataFrame(dflist,columns = ['M1','M2','M3','M4','M5','M6','M7','M8'] , index = ['W1','W2','W3','W4','W5','W6','W7','W8'])

W1->M2: 185.0
W2->M5: 350.0
W3->M3: 375.0
W3->M5: 30.0
W4->M2: 200.0
W4->M4: 70.0
W4->M8: 10.0
W5->M6: 100.0
W5->M7: 5.0
W6->M8: 75.0
W7->M1: 175.0
W7->M2: 15.0
W7->M3: 175.0
W7->M7: 60.0
W8->M5: 270.0


In [19]:
df

,M1,M2,M3,M4,M5,M6,M7,M8
W1,0.0,185.0,0.0,0.0,0.0,0.0,0.0,0.0
W2,0.0,0.0,0.0,0.0,350.0,0.0,0.0,0.0
W3,0.0,0.0,375.0,0.0,30.0,0.0,0.0,0.0
W4,0.0,200.0,0.0,70.0,0.0,0.0,0.0,10.0
W5,0.0,0.0,0.0,0.0,0.0,100.0,5.0,0.0
W6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75.0
W7,175.0,15.0,175.0,0.0,0.0,0.0,60.0,0.0
W8,0.0,0.0,0.0,0.0,270.0,0.0,0.0,0.0
